# 🎨 Prompt-to-ASCII Art Generator

This notebook provides an interactive experimentation environment for the ASCII art generator.

## Features
- **Image Generation**: Stable Diffusion (SDXL-Turbo) for fast text-to-image
- **Structural Mapping**: AISS (Log-Polar) and Random Forest algorithms
- **Multiple Charsets**: ASCII, ANSI blocks, Shift-JIS
- **Quality Metrics**: SSIM, edge preservation, diversity

## 1. Setup & Installation

In [ ]:
# Install dependencies (run once)
# !pip install -r ../requirements.txt

In [ ]:
import sys
sys.path.insert(0, '..')

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Import our library
from ascii_gen import PromptToASCII, get_charset
from ascii_gen.pipeline import image_to_ascii, prompt_to_ascii
from ascii_gen.charsets import list_charsets, visualize_charset
from ascii_gen.metrics import compute_ssim, character_diversity

print("✅ ASCII Generator loaded successfully!")

## 2. Explore Character Sets

In [ ]:
# List available charsets
print("Available character sets:")
for name in list_charsets():
    charset = get_charset(name)
    print(f"  • {name}: {len(charset.characters)} characters")

In [ ]:
# Visualize a charset
charset = get_charset("ascii_structural")
viz = visualize_charset(charset, cols=10)

plt.figure(figsize=(10, 4))
plt.imshow(viz, cmap='gray')
plt.title(f"Character Set: {charset.name}")
plt.axis('off')
plt.show()

## 3. Image-to-ASCII Conversion (No GPU Required)

In [ ]:
# Create a simple test image (or load your own)
# For demo, we'll create a gradient with shapes

import cv2

# Create test image with geometric shapes
test_img = np.ones((400, 600, 3), dtype=np.uint8) * 255

# Draw shapes
cv2.circle(test_img, (150, 200), 80, (0, 0, 0), 3)  # Circle
cv2.rectangle(test_img, (280, 120), (400, 280), (0, 0, 0), 3)  # Square
pts = np.array([[500, 280], [450, 120], [550, 120]], np.int32)  # Triangle
cv2.polylines(test_img, [pts], True, (0, 0, 0), 3)

test_image = Image.fromarray(test_img)

plt.figure(figsize=(10, 6))
plt.imshow(test_image)
plt.title("Test Image: Geometric Shapes")
plt.axis('off')
plt.show()

In [ ]:
# Convert to ASCII using Random Forest
result = image_to_ascii(
    test_image,
    mapper="random_forest",
    charset="ascii_structural",
    char_width=60,
)

print("Random Forest Result:")
print("=" * 60)
result.display()

In [ ]:
# Compare AISS vs Random Forest
pipeline = PromptToASCII(mapper="both", charset="ascii_structural")
aiss_result, rf_result = pipeline.from_image(
    test_image, 
    char_width=60,
    return_comparison=True
)

print("AISS Result:")
print("=" * 60)
aiss_result.display()

print("\nRandom Forest Result:")
print("=" * 60)
rf_result.display()

In [ ]:
# Compare quality metrics
comparison = pipeline.compare_results(aiss_result, rf_result)

print("\n📊 Quality Metrics Comparison:")
print(f"{'Metric':<20} {'AISS':>12} {'Random Forest':>15}")
print("-" * 50)
for metric in ['ssim', 'diversity', 'edge_preservation']:
    aiss_val = comparison['aiss'][metric]
    rf_val = comparison['random_forest'][metric]
    winner = "🏆" if rf_val > aiss_val else "" if aiss_val > rf_val else "="
    print(f"{metric:<20} {aiss_val:>12.4f} {rf_val:>12.4f} {winner}")

## 4. Prompt-to-ASCII with Stable Diffusion

⚠️ **Note**: This requires downloading SDXL-Turbo (~6GB) on first run.

In [ ]:
# Generate ASCII from prompt
# This will download the model on first run

prompt = "a simple cat face, line art, minimal"

result = prompt_to_ascii(
    prompt,
    mapper="random_forest",
    charset="ascii_standard",
    char_width=60,
    seed=42,
)

print(f"Prompt: '{prompt}'")
print("=" * 60)
result.display()

In [ ]:
# Show the source image alongside ASCII
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Source image
axes[0].imshow(result.source_image)
axes[0].set_title("Generated Image")
axes[0].axis('off')

# ASCII (rendered)
from ascii_gen.metrics import render_ascii_to_image
ascii_img = render_ascii_to_image(result.text, font_size=8)
axes[1].imshow(ascii_img, cmap='gray')
axes[1].set_title("ASCII Art (Rendered)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 5. Experiment with Different Charsets

In [ ]:
# Compare different character sets on the same image
charsets_to_test = ["ascii_standard", "ascii_dense", "ascii_structural", "ansi_blocks"]

for charset_name in charsets_to_test:
    try:
        result = image_to_ascii(
            test_image,
            mapper="random_forest",
            charset=charset_name,
            char_width=40,
        )
        print(f"\n{charset_name.upper()}:")
        print("-" * 40)
        result.display()
    except Exception as e:
        print(f"Error with {charset_name}: {e}")

## 6. Save Results

In [ ]:
# Save as text file
result.save("../examples/output.txt")
print("Saved to examples/output.txt")

# Save as HTML with styling
result.save("../examples/output.html")
print("Saved to examples/output.html")

## 7. Performance Benchmarks

In [ ]:
import time

# Benchmark AISS vs Random Forest
n_runs = 5

# AISS timing
pipeline_aiss = PromptToASCII(mapper="aiss", charset="ascii_structural")
aiss_times = []
for _ in range(n_runs):
    start = time.time()
    _ = pipeline_aiss.from_image(test_image, char_width=60)
    aiss_times.append(time.time() - start)

# RF timing
pipeline_rf = PromptToASCII(mapper="random_forest", charset="ascii_structural")
rf_times = []
for _ in range(n_runs):
    start = time.time()
    _ = pipeline_rf.from_image(test_image, char_width=60)
    rf_times.append(time.time() - start)

print("⏱️ Performance Comparison:")
print(f"AISS:          {np.mean(aiss_times)*1000:.1f}ms ± {np.std(aiss_times)*1000:.1f}ms")
print(f"Random Forest: {np.mean(rf_times)*1000:.1f}ms ± {np.std(rf_times)*1000:.1f}ms")
print(f"\nSpeedup: {np.mean(aiss_times)/np.mean(rf_times):.1f}x faster with Random Forest")

---

## Next Steps

1. **Try your own images**: Load any image with `Image.open('path/to/image.jpg')`
2. **Experiment with prompts**: Use Stable Diffusion to generate base images
3. **Tune parameters**: Adjust `char_width`, `tile_size`, edge detection thresholds
4. **Train custom RF model**: Use `pipeline.save_rf_model()` to save your trained model